### Introduction
Due to the limitations of AzureMLEndpoints for reusability on local environments (people without keys cannot access), it was considered feasible to try out Local Deployment of Llama2 to see whether the full modification of the code (by adding, not replacing) should be initiated. 

### Notebook contents
The notebook tests the capabilities of Llama2's 7B model for generation of XML in the LegalRuleML format according to the GPTQ quantised model available on HuggingFace and incorporated via LangChain.

### Results
There has been a limitation with respect to context-windows in other quantisation methods, which has apparently been bypassed with subtlety in the GPTQ quantised model. It has generated the XML fairly accurately in accordance with a single example. 

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import requests
import os
import urllib.request
import ssl
import json


In [5]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


model_name_or_path = "TheBloke/Llama-2-7B-Chat-GPTQ"
# To use a different branch, change revision
# For example: revision="main"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

/root/anaconda3/envs/Schematise-dev-server/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)
hf = HuggingFacePipeline(pipeline=pipe)

In [2]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

examples = [
    {
        "question": "Statute: Punishment for piracy - Whoever commits any act of piracy shall be punished with imprisonment for life or with fine or with both; or with death if the act of piracy causes death.",
        "answer": """
<LegalRuleML xmlns="http://www.oasis-open.org/committees/legalruleml">
  <lrml:PrescriptiveStatement id="PunishmentForPiracy">
    <lrml:Rule id="RuleForPiracyPunishment">
      <lrml:if>
      <lrml:Fact>
          <lrml:Rel iri="#commitPiracy"/>
        </lrml:Fact>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <lrml:Sanction>
            <lrml:Penalty>
              <lrml:Type iri="#imprisonment"/>
              <lrml:Duration max="life"/>
            </lrml:Penalty>
            <lrml:Condition>
              <lrml:CausesDeathOrAttempt/>
              <lrml:AdditionalSanctions>
                <lrml:Restitution/>
                <lrml:ForfeitureOfProperty/>
              </lrml:AdditionalSanctions>
            </lrml:Condition>
          </lrml:Sanction>
        </lrml:Obligation>
      </lrml:then>
    </lrml:Rule>
  </lrml:PrescriptiveStatement>
</LegalRuleML>"""}]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: Statute: Punishment for piracy - Whoever commits any act of piracy shall be punished with imprisonment for life or with fine or with both; or with death if the act of piracy causes death.
AI: 
<LegalRuleML xmlns="http://www.oasis-open.org/committees/legalruleml">
  <lrml:PrescriptiveStatement id="PunishmentForPiracy">
    <lrml:Rule id="RuleForPiracyPunishment">
      <lrml:if>
      <lrml:Fact>
          <lrml:Rel iri="#commitPiracy"/>
        </lrml:Fact>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <lrml:Sanction>
            <lrml:Penalty>
              <lrml:Type iri="#imprisonment"/>
              <lrml:Duration max="life"/>
            </lrml:Penalty>
            <lrml:Condition>
              <lrml:CausesDeathOrAttempt/>
              <lrml:AdditionalSanctions>
                <lrml:Restitution/>
                <lrml:ForfeitureOfProperty/>
              </lrml:AdditionalSanctions>
            </lrml:Condition>
          </lrml:Sanction>
       

In [3]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a consistent Legal Rule ML converter who takes the statute text as input and outputs the XML format for the same with reference to the documentation available at http://docs.oasis-open.org/legalruleml/legalruleml-core-spec/v1.0/legalruleml-core-spec-v1.0.html"),
        few_shot_prompt,
        ("human", "{question}"),
    ]
)

In [9]:
llm_chain = final_prompt | hf
section_for_conversion = """
6. Responsibilities of collection centres.	collect e-waste on behalf of producer or dismantler or recycler or refurbisher including those arising from orphaned products;, Provided the collection centres established by producer can also collect e-waste on behalf of dismantler, refurbisher and recycler including those arising from orphaned products, ensure that the facilities are in accordance with the standards or guidelines issued by Central Pollution Control Board from time to time;, ensure that the e-waste collected by them is stored in a secured manner till it is sent to authorised dismantler or recycler as the case may be;, ensure that no damage is caused to the environment during storage and transportation of e-waste;, maintain records in Form-2 of the e-waste handled as per the guidelines of Central Pollution Control Board and make such records available for scrutiny by the Central Pollution Control Board or the concerned State Pollution Control Board as and when asked for.
"""

section_for_conversion

'\n6. Responsibilities of collection centres.\tcollect e-waste on behalf of producer or dismantler or recycler or refurbisher including those arising from orphaned products;, Provided the collection centres established by producer can also collect e-waste on behalf of dismantler, refurbisher and recycler including those arising from orphaned products, ensure that the facilities are in accordance with the standards or guidelines issued by Central Pollution Control Board from time to time;, ensure that the e-waste collected by them is stored in a secured manner till it is sent to authorised dismantler or recycler as the case may be;, ensure that no damage is caused to the environment during storage and transportation of e-waste;, maintain records in Form-2 of the e-waste handled as per the guidelines of Central Pollution Control Board and make such records available for scrutiny by the Central Pollution Control Board or the concerned State Pollution Control Board as and when asked for.\n

In [11]:
print (llm_chain.invoke({"question": section_for_conversion}))

AI: 
<LegalRuleML xmlns="http://www.oasis-open.org/committees/legalruleml">
  <lrml:PrescriptiveStatement id="ResponsibilitiesOfCollectionCentres">
    <lrml:Rule id="RuleForCollectingEWaste">
      <lrml:if>
      <lrml:Fact>
          <lrml:Rel iri="#collectEWasteOnBehalfProducerDismantlerRecyclerRefurbisherIncludingOrphanedProducts"/>
        </lrml:Fact>
      </lrml:if>
      <lrml:then>
        <lrml:Obligation>
          <lrml:Directive>
            <lrml:Action>
              <lrml:EstablishFacilitiesInAccordanceWithStandardsIssuedByCentralPollutionControlBoardFromTimeToTime/>
            </lrml:Action>
          </lrml:Directive>
        </lrml:Obligation>
      </lrml:then>
    </lrml:Rule>
  </lrml:PrescriptiveStatement>
</LegalRuleML>
